In [1]:
import pandas as pd
import random

In [2]:
import requests
from bs4 import BeautifulSoup

## Telegram

In [3]:
# pip install telethon

In [23]:
from telethon.sync import TelegramClient

name = 'Paige Lee' 
api_id = '13544320' 
api_hash = '1029298096dbdcfc04c3c027a33c5eae' 
chat = '@metajournal'

data = [] # stores all our data in the format SENDER_ID, MSG

async with TelegramClient(name, api_id, api_hash) as client:
    async for message in client.iter_messages(chat):
        data.append({
            'senderUsername' : message.sender.username,
            'date' : message.date,
            'text' : message.text})

In [24]:
df = pd.DataFrame.from_records(data)
print(len(df))
df.head()

2264


,senderUsername,date,text
0,metajournal,2022-09-16 14:05:10+00:00,#переводы \nСергей Жадан\n\n+ + +\n\nЗапомнить...
1,metajournal,2022-09-15 19:57:48+00:00,"Стихотворение Лизы Смирновой - одно из тех, ко..."
2,metajournal,2022-09-15 15:34:36+00:00,Лиза Смирнова\n\n* * *\n\nоднажды в ваших жизн...
3,metajournal,2022-09-15 13:58:55+00:00,Визуальный комментарий к стихотворению Андрея ...
4,metajournal,2022-09-14 13:50:29+00:00,"В стихах, написанных этой весной и летом, Андр..."


In [25]:
df['date'] = df['date'].apply(lambda t : t.tz_convert(None))

In [27]:
mzRecs = df.to_dict('records')

In [91]:
texts = []
authors = []
perevods = []

newRecs = []
for r in mzRecs:
    if r['text'] == None:
        continue
    if r['text'].strip() == '':
        continue
    if r['text'].count('\n') < 2:
        continue
    if '#комментарий' in r['text']:
        continue
    if '#перевод' in r['text']:
        perevods.append(r)
        continue
    txt = r['text']
    a = txt.split('\n')[0]
    t = '\n'.join(txt.split('\n')[1:])
    newRecs.append({
        'date' : f"{r['date'].month}-{r['date'].day}-{r['date'].year}",
        'sender' : r['senderUsername'],
        'author' : a.replace('*',''),
        'text' : t
    })

print(len(whatsleft))
print(len(perevods))

1158
116


In [92]:
df = pd.DataFrame.from_records(newRecs)
print(len(df))
df.head()

1158


,date,sender,author,text
0,9-15-2022,metajournal,Лиза Смирнова,\n* * *\n\nоднажды в ваших жизнях \nничего не ...
1,9-14-2022,metajournal,Андрей Гришаев,"\n***\n\nЯ далеко зашёл, я шёл и далеко\nЗашёл..."
2,9-12-2022,metajournal,Дарья Андреева,\n\nМы выбрали школу с хорошим бомбоубежищем\n...
3,9-11-2022,metajournal,"Стихотворение украинского поэта, а также психо...",\nПриведу в связи с этим пару исчерпывающих ци...
4,9-11-2022,metajournal,Борис Херсонский,\n*****\nОтец пожал плечами: хочешь курить — к...


In [94]:
df.to_excel('MetazhurnalTelegramPosts9-17-2022.xlsx')

## No War Poetry

In [114]:
site = 'https://nowarpoetry.com/authors/'

soup = BeautifulSoup(requests.get(site).content, "html.parser")

In [121]:
hrefs = []
for a in soup.find_all('a')[4:]:
    if a.text == 'WordPress':
        break
    hrefs.append(a['href'])

In [216]:
allPoems = []

for url in hrefs:
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for br in soup.find_all("br"):
        br.replace_with("\n")

    poems = []
    ps = soup.find_all('p')
    newpoem = False
    poemlines = []
    for i, p in enumerate(ps):
        text = p.text
        if i == 0:
            author = p.text
            continue
        if p.text.strip() == '':
            continue
        if p.text == 'П О Э Т Ы':
            poems.append(('\n').join(poemlines))
            break
        if p.text[0] == '[' and ']' in p.text:
            print(p.text)
            continue
        if p.strong and poemlines != []:
            poems.append(('\n').join(poemlines))
            newpoem = True
            poemlines = []
            continue
        if ps[i-1].strong and ps[i-1].text.count('/') < 2 and i != 2:
            text = ps[i-1].text + '\n' + p.text
        text = text.replace('/   /   /','')
        poemlines.append(text + '\n')
    
    for poem in poems:
        allPoems.append({
            'Author' : author,
            'Text' : poem,
            'URL' : url,
            'Before or after' : 'After'
        })

[1] Ab ovo — в буквальном переводе «с яйца». Устойчивый фразеологический оборот, обозначающий «с самого начала».
[2] Плутон –в древнегреческой и римской мифологии
одно из имён бога подземного царства и смерти.
[3] Волока — единица измерения площади, равная 30 моргам или 20 десятинам (21,36 га), а также единица обложения в Великом княжестве Литовском.
[4] Морг – единица измерения площади земли в средневековой Западной Европе и, в частности, в Речи Посполитой, равная приблизительно 0,56 гектара.
[5] лат. Sero molunt deorum molae — «Поздно мелют мельницы богов» — крылатое выражение, означающее неотвратимость Судьбы.
[6] «№ 5, 1948» — картина Джексона Поллока, завершённая в 1948 году. Одна из самых известных работ в стиле абстрактного экспрессионизма.
[7] Отсылка к «Пушкин и Гоголь» Даниила Хармса.
[1] Арад — город на юге Израиля
[2] Один — верховный бог в германо-скандинавской мифологии
[3] Па́род (др.-греч. πάροδος) – в древнегреческом театре (трагедии и комедии) хоровая песня, которая и

In [217]:
len(allPoems)

794

In [218]:
random.choice(allPoems)

{'Author': 'АЛЕКСАНДР ДЕЛЬФИНОВ ',
 'Text': 'Война\n\n«Как ни печально признавать, порой война необходима,\nВедь после кризиса опять жизнь расцветёт, – сказал мне Дима. —\nВолна огня очистит почву, начнётся новая весна,\nНа всём плохом поставив точку – вот для чего нужна война!»\nМы с ним в кафе сидели где-то, не помню, в Питере, в Москве,\nНо это точно было лето, и ветерок шуршал в листве,\nИ я представил, как внезапно сметает всё волна огня,\nЗачистив землю поэтапно, и нет ни Димы, ни меня,\nИ только в бункере бетонном на генераторном ходу\nГигантский сервер «Амазона» гудит в густую пустоту.\nПрошла лишь пара-тройка лет,\nИ Димы правда больше нет.\n\nМы все в солдатики играли, мальчишки из СССР,\nИ пленных – нет, не убивали, простите, благородный сэр,\nПозвольте выйти с белым флагом навстречу вражеским войскам,\nВедь честным и лихим воякам почёт и слава тут и там,\nИ после битвы злой и страшной хотим, как водится в игре,\nМы наших обменять на ваших за гаражом на пустыре.\nХоть, помню

In [219]:
nwpDf = pd.DataFrame.from_records(allPoems)
nwpDf.head()

,Author,Text,URL,Before or after
0,АЛЕКСАНДР АМЧИСЛАВСКИЙ,СОЛДАТСКАЯ ПРОЩАЛЬНАЯ\n\nВо сне а будто бы в б...,https://nowarpoetry.com/authors/alexander-amch...,After
1,АЛЕКСАНДР АМЧИСЛАВСКИЙ,Серый лёд ни капли света\nгрязная вода\nотрабо...,https://nowarpoetry.com/authors/alexander-amch...,After
2,АЛЕКСАНДР АМЧИСЛАВСКИЙ,"Боишься, мальчик, сравнивать свой дом\nс могил...",https://nowarpoetry.com/authors/alexander-amch...,After
3,АЛЕКСАНДР АМЧИСЛАВСКИЙ,Нас тьмы и тьмы. И чёрная зима\nлежит раскинул...,https://nowarpoetry.com/authors/alexander-amch...,After
4,АНАСТАСИЯ АНДРЕЕВА,\n\nони не знали что их завтра ждет\nвойна пол...,https://nowarpoetry.com/authors/anastasia-andr...,After


In [220]:
nwpDf.to_excel('Excel_files/No_War_Poetry_9-19.xlsx')